# Strain mapping

This notebook provides an easy to use interface to the software package for the single Bragg edge fitting, for the analysis  of strain and texture variations.

Full step by step tutorial can be found [here](here)

# Importing python modules and functions

In [1]:
import numpy as np
from jupyter_notebooks.code.strain_mapping_api_for_notebook import StrainMappingAPIForNotebook
from jupyter_notebooks.code.roi_selection import Interface as RoiInterface
from jupyter_notebooks.code.fit_interface import Interface as FitInterface

ImportError: cannot import name 'Interface'

In [2]:
%gui qt

# General Settings 

In [3]:
o_api = StrainMappingAPIForNotebook()
o_api.general_settings()

# Loading sample, time spectra and open beam.

## Select sample folder

Select the folder(s) containing the sample data. 

If you are **working with raw data**, this/these folder(s) must contain:
 * all the projections
 * time spectra file  (\*_Spectra.txt)
 * shutter count (\*_ShutterCount.txt) **if working with RAW data**
 * shutter times (\*_ShutteTimes.txt)
 
If you are **not working with raw data**, the notebook will automatically load the **time spectra** file, if located in the same folder, otherwise it will ask you to select it.

**NB**:
- If more than 1 folder is selected, the folder will be combined using mean values.

In [4]:
o_api.select_data()

# Data Fitting

In [5]:
o_gui = RoiInterface(main_api=o_api)
o_gui.show()

# Data fitting
Data is now ready for fitting. We fit both using Gaussian method and Advanced method. In both case we test first on a single pixel by setting the debug_idx parameter, and we use the fit results as initial guess for the full image fitting to increase performance (both speed and quality).

In [6]:
o_fit = FitInterface(o_roi=o_gui,
                     o_api=o_api)
o_fit.show()

Data and filtering kernel are 3D. Applying 3D filter convolution.
